# Sample for KFServing SDK v1alpha2 

This is a sample for KFServing SDK v1alpha2. 

The notebook shows how to use KFServing SDK to create, get, rollout_canary, promote and delete InferenceService.

In [1]:
!pip install kfp-tekton
!pip install kfserving==0.4.1

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 85 kB 180 kB/s eta 0:00:01
     |████████████████████████████████| 55 kB 173 kB/s eta 0:00:01
     |████████████████████████████████| 88 kB 73 kB/s eta 0:00:011
     |████████████████████████████████| 1.5 MB 56 kB/s eta 0:00:01
     |████████████████████████████████| 47 kB 34 kB/s eta 0:00:01
  Attempting uninstall: kubernetes
    Found existing installation: kubernetes 11.0.0
    Uninstalling kubernetes-11.0.0:
      Successfully uninstalled kubernetes-11.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp-tekton 0.5.1rc1 requires kubernetes==11.0.0, but you have kubernetes 10.0.1 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [3]:
from kubernetes import client

from kfserving import KFServingClient
from kfserving import constants
from kfserving import utils
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TensorflowSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kubernetes.client import V1ResourceRequirements

In [4]:
constants.KFSERVING_VERSION

'v1alpha2'

Define namespace where InferenceService needs to be deployed to. If not specified, below function defines namespace to the current one where SDK is running in the cluster, otherwise it will deploy to default namespace.

In [2]:
namespace = utils.get_default_target_namespace()

## Define InferenceService

Firstly define default endpoint spec, and then define the inferenceservice basic on the endpoint spec.

In [3]:
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                            tensorflow=V1alpha2TensorflowSpec(
                              storage_uri='gs://kfserving-samples/models/tensorflow/flowers',
                              resources=V1ResourceRequirements(
                                  requests={'cpu':'100m','memory':'1Gi'},
                                  limits={'cpu':'100m', 'memory':'1Gi'}))))
    
isvc = V1alpha2InferenceService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(
                              name='flower-sample', namespace=namespace),
                          spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec))

In [ ]:
KFServing = KFServingClient()
KFServing.delete('flower-sample', namespace=namespace)

## Create InferenceService

Call KFServingClient to create InferenceService.

In [4]:
KFServing = KFServingClient()
KFServing.create(isvc)

ConfigException: Invalid kube-config file. No configuration found.

## Check the InferenceService

In [ ]:
KFServing.get('flower-sample', namespace=namespace, watch=True, timeout_seconds=120)

## Delete the InferenceService

In [ ]:
KFServing.delete('flower-sample', namespace=namespace)